In [ ]:
!pip3 -qq install torch
!pip -qq install torchtext
!pip -qq install torchvision
!pip -qq install spacy
!python -m spacy download en
!pip install sacremoses
!pip install subword_nmt
!wget -qq http://www.manythings.org/anki/rus-eng.zip 
!unzip rus-eng.zip

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Machine Translation

Мы уже несколько раз смотрели на эту картинку:

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg)

*From [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*

В POS tagging мы (ну, некоторые точно) использовали важную идею: сначала некоторой функцией над символами строился эмбеддинг для слова (например, many to one rnn'кой на картинке). Потом другая rnn'ка строила эмбеддинги слов с учетом их контекста. А дальше это всё классифицируется логистической регрессией.

Тут важно то, что мы обучаем encoder для построения эмбеддингов end2end - прямо в составе сети (это основное отличие нейросетей от классических подходов - в умении делать end2end).

Другое, что мы делали - это языковые модели. Вот, типа такого:
![](https://hsto.org/web/dc1/7c2/c4e/dc17c2c4e9ac434eb5346ada2c412c9a.png)

Обратите внимание на красную стрелку - она показывает передачу скрытого состояния, которое отвечает за память сети.

А теперь совместим эти две идеи:

![](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/seq2seq.jpg)  
*From [tensorflow/nmt](https://github.com/tensorflow/nmt)*

Выглядит все почти как языковая модель, но в синей части предсказания не делаются, используется только последнее скрытое состояние.

Синяя часть сети называется энкодер, она строит эмбеддинг последовательности. Красная часть - декодер, работает как обычная языковая модель, но учитывает результат работы энкодера.

В итоге, энкодер учится эффективно извлекать смысл из последовательности слов, а декодер должен строить по ним новую последовательность. Это может быть последовательностью слов перевода, или последовательностью слов в ответе чат бота, или еще чем-то в зависимости от вашей испорченности.

## Подготовка данных

Начнем с чтения данных. Возьмем их у anki, поэтому они немного специфичны:

In [ ]:
!shuf -n 10 rus.txt

Токенизируем их:

In [ ]:
from torchtext.data import Field, Example, Dataset, BucketIterator

BOS_TOKEN = '<s>'
EOS_TOKEN = '</s>'

source_field = Field(tokenize='spacy', init_token=None, eos_token=EOS_TOKEN)
target_field = Field(tokenize='moses', init_token=BOS_TOKEN, eos_token=EOS_TOKEN)
fields = [('source', source_field), ('target', target_field)]

In [ ]:
source_field.preprocess("It's surprising that you haven't heard anything about her wedding.")

In [ ]:
target_field.preprocess('Удивительно, что ты ничего не слышал о её свадьбе.')

In [ ]:
from tqdm import tqdm

MAX_TOKENS_COUNT = 16
SUBSET_SIZE = .3

examples = []
with open('rus.txt') as f:
    for line in tqdm(f):
        source_text, target_text, _ = line.split('\t')
        source_text = source_field.preprocess(source_text)
        target_text = target_field.preprocess(target_text)
        if len(source_text) <= MAX_TOKENS_COUNT and len(target_text) <= MAX_TOKENS_COUNT:
            if np.random.rand() < SUBSET_SIZE:
                examples.append(Example.fromlist([source_text, target_text], fields))

Построим датасеты:

In [ ]:
dataset = Dataset(examples, fields)

train_dataset, test_dataset = dataset.split(split_ratio=0.85)

print('Train size =', len(train_dataset))
print('Test size =', len(test_dataset))

source_field.build_vocab(train_dataset, min_freq=3)
print('Source vocab size =', len(source_field.vocab))

target_field.build_vocab(train_dataset, min_freq=3)
print('Target vocab size =', len(target_field.vocab))

train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), batch_sizes=(32, 256), shuffle=True, device=DEVICE, sort=False
)

In [ ]:
source_field.process([source_field.preprocess("It's surprising that you haven't heard anything about her wedding.")])

In [ ]:
source_field.vocab.itos

In [ ]:
target_field.vocab.itos

## Seq2seq модель

Пора писать простой seq2seq. Разобьем модель на несколько модулей - Encoder, Decoder и их объединение. 

Encoder должен быть подобен символьной сеточке в POS tagging'е: эмбеддить токены и запускать rnn'ку (в данном случае будем пользоваться GRU) и отдавать последнее скрытое состояние.

Decoder почти такой же, только еще и предсказывает токены на каждом своем шаге.

**Задание** Реализовать модели.

In [ ]:
batch = next(iter(train_iter))

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1, bidirectional=False):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=rnn_hidden_dim, 
                           num_layers=num_layers, bidirectional=bidirectional)

    def forward(self, inputs, hidden=None):
        """
        input: LongTensor with shape (encoder_seq_len, batch_size)
        hidden: FloatTensor with shape (1, batch_size, rnn_hidden_dim)
        """
        # PASS CODE

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, rnn_hidden_dim=256, num_layers=1):
        super().__init__()

        self._emb = nn.Embedding(vocab_size, emb_dim)
        self._rnn = nn.GRU(input_size=emb_dim, hidden_size=rnn_hidden_dim, num_layers=num_layers)
        self._out = nn.Linear(rnn_hidden_dim, vocab_size)

    def forward(self, inputs, encoder_output, hidden=None):
        """
        input: LongTensor with shape (decoder_seq_len, batch_size)
        encoder_output: FloatTensor with shape (encoder_seq_len, batch_size, rnn_hidden_dim)
        encoder_mask: ByteTensor with shape (encoder_seq_len, batch_size) (ones in positions of <pad> tokens, zeros everywhere else)
        hidden: FloatTensor with shape (1, batch_size, rnn_hidden_dim)
        """
        # PASS CODE

Модель перевода будет просто сперва вызывать Encoder, а потом передавать его скрытое состояние декодеру в качестве начального.

In [ ]:
class TranslationModel(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, emb_dim=128, 
                 rnn_hidden_dim=256, num_layers=1, bidirectional_encoder=False):
        super().__init__()
        
        self.encoder = Encoder(source_vocab_size, emb_dim, rnn_hidden_dim, num_layers, bidirectional_encoder)
        self.decoder = Decoder(target_vocab_size, emb_dim, rnn_hidden_dim, num_layers)
        
    def forward(self, source_inputs, target_inputs=None):
        encoder_output, encoder_hidden = self.encoder(source_inputs)
        
        return self.decoder(target_inputs, encoder_hidden, encoder_hidden)

In [ ]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), 
                         target_vocab_size=len(target_field.vocab)).to(DEVICE)

out, _ = model(batch.source.to(DEVICE), batch.target.to(DEVICE))
print(out.shape, batch.source.shape, batch.target.shape)

Реализуем простой перевод - жадный. На каждом шаге будем выдавать самый вероятный из предсказываемых токенов:

![](https://github.com/tensorflow/nmt/raw/master/nmt/g3doc/img/greedy_dec.jpg)  
*From [tensorflow/nmt](https://github.com/tensorflow/nmt)*

**Задание** Реализовать функцию.

In [ ]:
def greedy_decode(model, source_text, source_field, target_field):
    bos_index = target_field.vocab.stoi[BOS_TOKEN]
    eos_index = target_field.vocab.stoi[EOS_TOKEN]
    
    model.eval()
    with torch.no_grad():
        result = []

        # PASS CODE
        
        return ' '.join(target_field.vocab.itos[ind] for ind in result)

In [ ]:
greedy_decode(model, "Do you believe?", source_field, target_field)

Нужно как-то оценивать модель.

Обычно для этого используется [BLEU скор](https://en.wikipedia.org/wiki/BLEU) - что-то вроде точности угадывания n-gram из правильного (референсного) перевода.

**Задание** Реализовать функцию оценки: для батчей из `iterator` предсказать их переводы, обрезать по `</s>` и сложить правильные варианты и предсказанные в `refs` и `hyps` соответственно.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def evaluate_model(model, iterator):
    model.eval()
    refs, hyps = [], []
    bos_index = iterator.dataset.fields['target'].vocab.stoi[BOS_TOKEN]
    eos_index = iterator.dataset.fields['target'].vocab.stoi[EOS_TOKEN]
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            encoder_output, encoder_hidden = model.encoder(batch.source)
            
            # PASS CODE
            
    return corpus_bleu([[ref] for ref in refs], hyps) * 100

In [ ]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(model, criterion, data_iter, optimizer=None, name=None):
    epoch_loss = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = len(data_iter)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):                
                logits, _ = model(batch.source, batch.target)
                
                target = torch.cat((batch.target[1:], batch.target.new_ones((1, batch.target.shape[1]))))
                loss = criterion(logits.view(-1, logits.shape[-1]), target.view(-1))

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), 1.)
                    optimizer.step()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(name, loss.item(), 
                                                                                         math.exp(loss.item())))
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, PPX = {:.2f}'.format(
                name, epoch_loss / batches_count, math.exp(epoch_loss / batches_count))
            )
            progress_bar.refresh()

    return epoch_loss / batches_count


def fit(model, criterion, optimizer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss = do_epoch(model, criterion, train_iter, optimizer, name_prefix + 'Train:')
        
        if not val_iter is None:
            val_loss = do_epoch(model, criterion, val_iter, None, name_prefix + '  Val:')
            print('\nVal BLEU = {:.2f}'.format(evaluate_model(model, val_iter)))

In [ ]:
model = TranslationModel(source_vocab_size=len(source_field.vocab), target_vocab_size=len(target_field.vocab)).to(DEVICE)

pad_idx = target_field.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(DEVICE)

optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_iter, epochs_count=30, val_iter=test_iter)

In [ ]:
greedy_decode(model, "Do you want to believe ?", source_field, target_field)

## Scheduled Sampling

До сих пор мы тренировали перевод, используя так называемый *teacher forcing*: в качестве выхода на предыдущем шаге декодер принимал всегда правильный токен. Проблема такого подхода - во время инференса правильный токен, скорее всего, не выберется хотя бы на каком-то шаге. Получится, что сеть училась на хороших входах, использоваться будет на плохом - это легко может всё поломать.

Альтернативный подход - прямо во время обучения сэмплировать токен с текущего шага и передавать его на следующий.

Такой подход не слишком-то обоснован математически (градиенты не прокидываются через сэмплирование), но его интересно реализовать и он зачастую улучшает качество.

**Задание** Обновите `Decoder`: замените вызов rnn'ки над последовательностью на цикл. На каждом шаге вероятностью `p` передавайте в качестве предыдущего выхода декодеру правильный вход, а иначе - argmax от предыдущего выхода (цикл должен быть похожим на те, которые есть в `greedy_decode` и `evaluate_model`). При передаче argmax вызывайте `detach`, чтобы градиенты не прокидывались. Все выходы собирайте в список, в конце сделайте `torch.cat`. 

В результате при вероятности, равной `p=1`, должно получиться как раньше, только медленнее. При обучении можно передавать `p=0.5`, на инференсе - `p=1`.

## Улучшения модели

**Задание** Попробуйте повысить качество модели. Попробуйте: 
- Bidirectional encoder
- Dropout
- Stack moar layers

# Дополнительные материалы

## Статьи
Sequence to Sequence Learning with Neural Networks, Ilya Sutskever, et al, 2014 [[pdf]](https://arxiv.org/pdf/1409.3215.pdf)  
Show and Tell: A Neural Image Caption Generator, Oriol Vinyals et al, 2014 [[arxiv]](https://arxiv.org/abs/1411.4555)  
Scheduled Sampling for Sequence Prediction with Recurrent Neural Networks, Samy Bengio, et al, 2015 [[arxiv]](https://arxiv.org/abs/1506.03099)  
Neural Machine Translation of Rare Words with Subword Units, Rico Sennrich, 2015 [[arxiv]](https://arxiv.org/abs/1508.07909)  
Massive Exploration of Neural Machine Translation Architectures, Denny Britz, et al, 2017 [[pdf]](https://arxiv.org/pdf/1703.03906.pdf)

## Блоги
Neural Machine Translation (seq2seq) Tutorial [tensorflow/nmt](https://github.com/tensorflow/nmt)  
[A Word of Caution on Scheduled Sampling for Training RNNs](https://www.inference.vc/scheduled-sampling-for-rnns-scoring-rule-interpretation/)

## Видео
cs224n, [Machine Translation, Seq2Seq and Attention](https://www.youtube.com/watch?v=IxQtK2SjWWM)

## Разное
[The Annotated Encoder Decoder](https://bastings.github.io/annotated_encoder_decoder/)  
[Seq2Seq-Vis: A Visual Debugging Tool for Sequence-to-Sequence Models](http://seq2seq-vis.io)